In [1]:
!python --version

Python 3.9.12


In [ ]:
!pip install pandas
!pip install konlpy
!pip install umap
!pip install -U scikit-learn
!pip install --upgrade pip
!pip install matplotlib
!pip3 install plotly

In [1]:
from datetime import datetime

import logging
import re

import numpy as np
import pandas as pd

In [2]:
import os
os.getcwd()
import pandas as pd
path = "C:/Users/User-Pc/BerTopic/data/"
file_list = os.listdir(path)
dfs = []

In [3]:
def data_pro(data,from_date,to_date):

    df = data[['일자','키워드']]
    df['일자'] = [str(x)[0:6] for x in df['일자']] 

    datelist = pd.date_range(start=from_date, end=to_date, freq = 'M').tolist()
    dtlst = []

    for d_t in datelist:
        d_t = str(d_t)[0:-9]
        d = datetime.strptime(d_t, '%Y-%m-%d')
        d = d.strftime('%Y%m')
        dtlst.append(d)

    # 키워드 사전 
    keyword_vocab = []
    for i in df['키워드']:
        keyword_vocab.extend(i)
    keyword_vocab = list(set(keyword_vocab))

    # 월별 카운팅 후 병합
    from collections import Counter

    tmp_df_list = []
    for dt in dtlst:
        tmp = []
        for i in df[df['일자']== dt]['키워드']:
            tmp.extend(i)
        tmp_df = pd.DataFrame(pd.Series(Counter(tmp)))
        tmp_df = tmp_df.reset_index()
        tmp_df = tmp_df.rename(columns = {0:'cnt','index':'keyword'})
        tmp_df_list.append(tmp_df)

    df = tmp_df_list[0].merge(tmp_df_list[1], on = 'keyword', how = 'outer')
    for i in range(len(tmp_df_list)):
        if i > 1:
            df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
    col = ['keyword'] + dtlst
    df.columns = col

    # 합산 값 만들기
    df.index = df['keyword']
    df = df.drop('keyword', axis = 1)
    df['tot_cnt'] = df.sum(axis = 1)
    df = df.T
    df['year_cnt'] = df.sum(axis = 1)
    df = df.T
    df

    # 결측치 처리
    df =df.fillna(0)
    return df

In [4]:
for i in file_list:
    df = pd.read_excel(path+i)
    dfs.append(df)
    print(i)

C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20180101-20181213.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20190101-20191231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20200101-20201231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20210101-20211231.xlsx


C:\Users\User-Pc\miniconda3\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


NewsResult_20220101-20220812.xlsx


In [5]:
data = pd.concat(dfs)

In [6]:
## 데이터 전처리
data = pd.concat(dfs)
data = data[['뉴스 식별자','일자','제목','키워드','특성추출(가중치순 상위 50개)']]
cluster_mode = '특성추출(가중치순 상위 50개)'
corpus = data[cluster_mode].values.tolist()

df = data[['일자']]

for i, cp in enumerate(corpus):
    corpus[i] = cp.split(',')
df['키워드'] = corpus

df_k = data_pro(df,'20180801','20220731')

<ipython-input-6-233fab3f1b12>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['키워드'] = corpus
<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a Merg

In [7]:
data_e = pd.read_csv('C:/Users/User-Pc/Desktop/news_english.csv')

In [8]:
data_e = data_e.rename({'kwd':'키워드'},axis = 1)

In [9]:
tmp = []
for i in data_e['키워드']:
    i = i.replace('[','').replace(']','').replace('\'','').replace(' ','').split(',')
    tmp.append(i)

In [10]:
df_e = data_e[['일자','키워드']]
df_e['키워드'] = tmp

<ipython-input-10-53f16d203765>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_e['키워드'] = tmp


In [11]:
df_e = data_pro(df_e,'20190801','20220731')

<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
<ipython-input-3-c3ba82095b7f>:37: FutureWarning: Passing 'suffixes' which cause duplicate columns {'cnt_x'} in the result is deprecated and will raise a MergeError in a future version.
  df = df.merge(tmp_df_list[i], on = 'keyword', how = 'outer')
<ipy

In [19]:
# 데이터 확인
key = 'metaverse'
df_e[df_e.index == key]

,201908,201909,201910,201911,201912,202001,202002,202003,202004,202005,...,202111,202112,202201,202202,202203,202204,202205,202206,202207,tot_cnt
keyword,,,,,,,,,,,,,,,,,,,,,
metaverse,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,11.0,8.0,7.0,16.0,17.0,15.0,11.0,17.0,69.0,188.0


In [13]:
df_k = df_k[df_k['tot_cnt'] > 6]

In [14]:
df_e = df_e[df_e['tot_cnt'] > 10]

In [16]:
# 데이터 비교1 . 회귀선의 기울기
# 연도별 건수 보정(normaliae)

def get_slop(df2,from_date,to_date):
    datelist = pd.date_range(start=from_date, end=to_date, freq = 'M').tolist()
    dtlst = []
    df = df2.copy()
    for d_t in datelist:
        d_t = str(d_t)[0:-9]
        d = datetime.strptime(d_t, '%Y-%m-%d')
        d = d.strftime('%Y%m')
        dtlst.append(d)
    for dt in dtlst:
        df[dt] = df[dt][:-1]*10000 / df[dt][-1]
    from scipy import stats

    x = [x + 1for x in range(0,len(dtlst))]

    slopes = []
    for i in list(df.values):
        y = i[:-1]
        slope, intercept, r, p, std_err = stats.linregress(x, y)
        slopes.append(slope)
    df['slope'] = slopes
    return df

In [20]:
df_ks = get_slop(df_k,'20180801','20220731')

In [21]:
df_es = get_slop(df_e,'20190801','20220731')

In [22]:
import plotly.io as pio
import plotly.express as px

In [23]:
def Df4plot(df3,srch_wrd):
    df3 = df3.iloc[:,:-2]
    df3 = df3.loc[srch_wrd,:].reset_index()
    df3 = pd.melt(df3, id_vars='keyword', value_vars=list(df3))
    df3['variable'] = [x[:4] + '-' + x[4].replace('0','') + x[-1:] for x in df3['variable']]
    df3["variable"] = pd.to_datetime(df3.variable)
    df3.rename(columns={"variable": "Date"}, inplace=True)
    return df3

In [86]:
srch_wrd = ['메타버스','가상자산','가상통화','블록체인','빅테크','핀테크','nft','esg','마이데이터','mz',
        '코로나19','가상화폐','공인인증서','오픈뱅킹','인공지능','디지털자산','암호화폐','공동인증서','cbdc']
# 꺽은선 그래프
df3 = Df4plot(df_ks,srch_wrd)
fig = px.line(
    df3, x="Date", y='value', color="keyword", title="Key Word (2018.1 - 2022.7)"
)
fig.show()

In [90]:
# 꺽은선 그래프
df3 = Df4plot(df_k,['cbdc'])
fig = px.line(
    df3, x="Date", y='value', color="keyword", title="Key Word (2018.1 - 2022.7)"
)
fig.show()

In [69]:
key_k = '메타버스'
key_e = 'metaverse'

def keyCmp(key_k,key_e,df_ks,df_es):
    df_plot_k = Df4plot(df_ks,[key_k])
    df_plot_e = Df4plot(df_es,[key_e])

    df_plot_k['value'] = df_plot_k['value'] / sum(df_plot_k['value'])
    df_plot_e['value'] = df_plot_e['value'] / sum(df_plot_e['value'])
    if key_k == key_e:
        df_plot_e['keyword'] = df_plot_e['keyword'] + '(en)' 
    df_plot = pd.concat([df_plot_e,df_plot_k])
    fig = px.line(
        df_plot, x="Date", y='value', color="keyword", title="{} VS {}".format(key_k,key_e)
    )
    fig.show()


In [75]:
key_k = '메타버스'
key_e = 'metaverse'
keyCmp(key_k,key_e,df_ks.iloc[:,12:],df_es)

In [76]:
key_k = 'nft'
key_e = 'nft'
keyCmp(key_k,key_e,df_ks.iloc[:,12:],df_es)

In [77]:
key_k = 'cbdc'
key_e = 'cbdc'
keyCmp(key_k,key_e,df_ks.iloc[:,12:],df_es)

In [40]:
def mekko(df_k,srch_wrd):
    df3 =Df4plot(df_k,srch_wrd)
    df3['Percent'] = df3.groupby(['Date'])['value'].transform(lambda x: x/x.sum())
    fig = px.bar(df3, x="Date", y="Percent", color="keyword", text="keyword")
    fig.update_layout(bargap =0)
    fig.show()

In [ ]:
# 2020년 3월 5일, 대한민국에서 특정 금융거래정보의 보고 및 이용 등에 관한 법률 개정안 통과
# 특금법 개정안은 ▲암호화폐, 가상화폐, 디지털화폐 등 다양하게 사용되던 용어를 '가상자산(Virtual Assets)'으로 통일하

In [92]:
mekko(df_k,['가상자산','가상통화','암호화폐'])

In [45]:
mekko(df_e,['virtualassets','virtualcurrency','cryptocurrency'])

In [82]:
# 빅테크, 테트핀 라는 용어는 외국에서 디지털 금융 관련 키워드로 등장 빈도가 낮음.

mekko(df_k,['핀테크','빅테크','테크핀'])

In [84]:
mekko(df_e,['fintech','bigtech'])

In [14]:

'''
# 시각화 비교 남는 시간 방법
x = []
y = []
keys = []
min_cnt = 300
for i in keyword_vocab:
    word = i
    if df['2021'][word] > min_cnt and df['2022'][word] > min_cnt:
        conf_2021 = df['2021'][word]/df['2021'][-1]
        conf_2022 = df['2022'][word]/df['2022'][-1]
        ratio_2022 = conf_2022 / conf_2021
        ratio_2021 = conf_2021 / conf_2022
        key = word
        x.append(ratio_2021)
        y.append(ratio_2022)
        keys.append(key)
df3 = pd.DataFrame({'key2021':x,'key2022':y}, index = keys)
df3 = df3.dropna()
import numpy as np
df3 = df3.dropna()
df3 = df3[df3.sum(axis = 1) != np.inf]
df2.sum(axis = 1)[0]
df3.sort_values(by = 'key2021',ascending = False).head(10)
df3.sort_values(by = 'key2022',ascending = False).head(10)
# 데이터 확인
key = '농협은행'
df[df.index == key]
'''